In [1]:
import os
import pydicom
import pandas as pd
import numpy as np
import keras
import cv2

Using TensorFlow backend.


In [17]:
DICOMpath='D:/brain_CT'
other=['Batch 1', 'Batch 2','Batch 3','Batch 4','Topogram  1.0  T20s','<MPR Range>','<MIP Thin Range>','<MIP Thin Range[1]>','<MIP Thin Range[2]>','<MIP Thin Range[3]>', '<VRT Collection>', 'Orbit  3.0  H30s', '<MIP Thick Range>']
otherlist = []
for (path, dir, files) in os.walk(DICOMpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            metadata=pydicom.filereader.dcmread(filePath)
            if metadata.Modality=='CT' and metadata.BodyPartExamined.lower()=='head' and metadata.SeriesDescription in other:
                otherlist.append(filePath)

In [20]:
DICOMpath='D:/class/brainCTpost'
postlist = []
for (path, dir, files) in os.walk(DICOMpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            postlist.append(filePath)

In [21]:
DICOMpath='D:/class/brainCTnon'
nonlist = []
for (path, dir, files) in os.walk(DICOMpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            nonlist.append(filePath)

In [32]:
nondf=pd.DataFrame({'filename' : nonlist, 'label' : 0})
postdf=pd.DataFrame({'filename' : postlist, 'label' : 1})
otherdf=pd.DataFrame({'filename' : otherlist, 'label' : 2})
df=nondf.append(postdf)
df=df.append(otherdf)
df=df.sample(frac=1)

In [37]:
y_train = np.array(df['label'].tolist())
y_train = keras.utils.np_utils.to_categorical(y_train)

In [ ]:
x_train=[]
for i in df['filename'].values:
    a=pydicom.filereader.dcmread(str(i)).pixel_array
    if len(a)==2:
        a=cv2.resize(a, (128, 128))
    if len(a)==3:
        a=cv2.resize(a, (128, 128, 3))
    x_train.append(a)
x_train=np.array(x_train)  